### IMPORTS  FOR SKLEARN MODEL AND NLP LIBRARIES

In [1]:
import os
import nltk
import pandas as pd
import random
import statistics
import timeit
import itertools
from apyori import apriori
import pickle
import re
import string
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from config import DATABASE_FOLDER, DATABASE_FILE, FILE_JAR, FILE_MODELS_JAR
import warnings

### LOADING AND CLEANING DATASET 

In [2]:
warnings.filterwarnings("ignore")
java_path = "C:/Program Files/Java/jdk1.8.0_161/bin/java.exe"
os.environ['JAVAHOME'] = java_path
coreNLP_path = "./stanford-corenlp-3.9.2.jar"
os.environ["CORENLP_HOME"] = coreNLP_path

if __name__ == '__main__':
   
    pass
#setting target
TARGET           = ["Sentiment_positive"]

FEATURES         = list()
EXPLICIT_ASPECTS = set()
IMPLICIT_ASPECTS = set()

DATASET = DATABASE_FOLDER + DATABASE_FILE
data = pd.read_csv("full-corpus STS1.csv")
print(data.head())

  Sentiment  TweetId    Topic  \
0  positive        4  kindle2   
1  positive        5  kindle2   
2  positive        6  kindle2   
3  positive        7  kindle2   
4  positive        8  kindle2   

                                           TweetText  
0  Reading my kindle2...  Love it... Lee childs i...  
1  Ok, first assesment of the #kindle2 ...it fuck...  
2  @kenburbary You'll love your Kindle2. I've had...  
3  @mikefish  Fair enough. But i have the Kindle2...  
4  @richardebaker no. it is too big. I'm quite ha...  


In [3]:
# Define a function to clean the text
def clean(text):
# Removes all special characters and numericals leaving the alphabets
    text = re.sub('[^A-Za-z]+', ' ', text)
    text = re.sub(r"\W", " ", str(text))
    text = re.sub(r"\s+[a-zA-Z]\s+", " ", text)
    text = re.sub(r"\^[a-zA-Z]\s+", " ", text) 
    text = re.sub(r"\s+", " ", text, flags = re.I)
    text = re.sub(r"^b\s+", "", text)
    text = text.lower()
    for ch in string.punctuation:
        text = text.replace(ch, "")
    return text
    text = [word for word in text.split() 
            if word not in stopwords.words("english")]
    text = " ".join(text)

# Cleaning the text in the TweetText column
data['Cleaned TweetText'] = data['TweetText'].apply(clean)
data.head()

,Sentiment,TweetId,Topic,TweetText,Cleaned TweetText
0,positive,4,kindle2,Reading my kindle2... Love it... Lee childs i...,reading my kindle love it lee childs is good r...
1,positive,5,kindle2,"Ok, first assesment of the #kindle2 ...it fuck...",ok first assesment of the kindle it fucking ro...
2,positive,6,kindle2,@kenburbary You'll love your Kindle2. I've had...,kenburbary you ll love your kindle ve had min...
3,positive,7,kindle2,@mikefish Fair enough. But i have the Kindle2...,mikefish fair enough but have the kindle and ...
4,positive,8,kindle2,@richardebaker no. it is too big. I'm quite ha...,richardebaker no it is too big m quite happy ...


In [4]:
   # Lemmatization.
def clean_text(text: str, stem = True) -> str:
    if stem:
        stemmer = WordNetLemmatizer()
        text    = text.split()
        text    = [stemmer.lemmatize(word) for word in text]
        text    = " ".join(text)

    return text

In [6]:
# POS tagger dictionary and tokens
pos_dict = {'J':wordnet.ADJ, 'V':wordnet.VERB, 'N':wordnet.NOUN, 'R':wordnet.ADV}
def token_stop_pos(text):
    tags = pos_tag(word_tokenize(text))
    return tags

In [7]:
data['POS tagged and tokens'] = data['Cleaned TweetText'].apply(token_stop_pos)
data.head()

,Sentiment,TweetId,Topic,TweetText,Cleaned TweetText,POS tagged and tokens
0,positive,4,kindle2,Reading my kindle2... Love it... Lee childs i...,reading my kindle love it lee childs is good r...,"[(reading, VBG), (my, PRP$), (kindle, NN), (lo..."
1,positive,5,kindle2,"Ok, first assesment of the #kindle2 ...it fuck...",ok first assesment of the kindle it fucking ro...,"[(ok, NN), (first, JJ), (assesment, NN), (of, ..."
2,positive,6,kindle2,@kenburbary You'll love your Kindle2. I've had...,kenburbary you ll love your kindle ve had min...,"[(kenburbary, NN), (you, PRP), (ll, VBP), (lov..."
3,positive,7,kindle2,@mikefish Fair enough. But i have the Kindle2...,mikefish fair enough but have the kindle and ...,"[(mikefish, JJ), (fair, JJ), (enough, RB), (bu..."
4,positive,8,kindle2,@richardebaker no. it is too big. I'm quite ha...,richardebaker no it is too big m quite happy ...,"[(richardebaker, NN), (no, DT), (it, PRP), (is..."
